In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import requests
import json
import re

In [ ]:
def scrape_wikipedia(topic):
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic}"
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract the relevant information
    summary = data["extract"]
    return summary.lower()

In [ ]:
topic = input("Enter a topic to scrape: ")
summary = scrape_wikipedia(topic)
print("Summary:", summary)

In [ ]:
data = [{"summary":summary}]
df = pd.DataFrame(data, columns=["summary"])
df.head()

In [ ]:
df = re.sub(r'\[[0-9]*\]', ' ', str(df))
df = re.sub(r'\s+', ' ', str(df))

In [ ]:
df_sentences = nltk.sent_tokenize(df)
df_words = nltk.word_tokenize(df)

In [ ]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [ ]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup buddy","good night","I'm also good")
greeting_responses = ["hey", "hey, how are you?", "Yo Bro", "hello, how you doing", "hello", "Welcome, I am good and you"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def generate_response(user_input):
    bot_response = ''
    df_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(df_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        bot_response = bot_response + "I am sorry, I could not understand you"
        return bot_response
    else:
        bot_response = bot_response + df_sentences[similar_sentence_number]
        return bot_response

In [ ]:
word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(df_sentences)

In [ ]:
similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

In [ ]:
similar_sentence_number = similar_vector_values.argsort()[0]

In [ ]:
continue_dialogue = True
print("Hello, I am your Bot. You can ask me any question. And if you want to quit just type bye")
while(continue_dialogue == True):
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            print("Bot: Most welcome")
        else:
            if generate_greeting_response(human_text) != None:
                print("Bot: " + generate_greeting_response(human_text))
            else:
                print("Bot: ", end="")
                print(scrape_wikipedia(human_text))
    else:
        continue_dialogue = False
        print("Bot: Good bye and take care of yourself...")